# Projet IA

In [ ]:
import scipy
import numpy
import math
import sklearn
import pandas as pds
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import sergio_peignier as sp

## Attribute selection

In [ ]:
sparse_df = pds.read_csv("./datas/Data-IA-World-Development-Indicator.txt", sep="\t", header=0)

In [ ]:
sparse_df.shape

In [ ]:
sparse_df.head()

In [ ]:
sparse_df.tail()

In [ ]:
# Un peu de statistiques :
sparse_df.describe()

In [ ]:
#Fonction qui supprime une colonne (attribut) si le nb de NaN >= limit

def delete_NaN_col(df, limit):
    NaN_col = df.isna().sum()
    tmp = df.copy(deep=True) #temporary df
    
    for i in range(df.shape[1]):
        if NaN_col[i] >= limit:
            df = df.drop(columns = tmp.columns[i])
            
    return(df)

In [ ]:
sparse_df2 = delete_NaN_col(sparse_df, 40)

In [ ]:
# On enlèvre les colonnes 'Time' et 'Time Code' qui ne nous interessent pas ici
sparse_df2 = sparse_df2.drop(columns = ['Time', 'Time Code', 'Country Code'])

In [ ]:
sparse_df2

In [ ]:
sparse_df2.isna().sum()

In [ ]:
sparse_df2.isna().sum(axis=1)

In [ ]:
#Fonction qui supprime une ligne (object) si le nb de NaN >= limit

def delete_NaN_row(df, limit):
    NaN_row = df.isna().sum(axis=1)
    tmp = df.copy(deep=True) #temporary df
    
    for i in range(df.shape[0]):
        if NaN_row[i] >= limit:
            df = df.drop([i], axis = 0)
            
    return(df)

In [ ]:
sparse_df3 = delete_NaN_row(sparse_df2, 1)

In [ ]:
sparse_df3

In [ ]:
# Nous allons maintenant stocker les noms de pays dans la variable country_labels
# En effet, nous pourrons supprimer la colonne "Country Name" et n'avoir plus que des valeurs numériques.
# Les noms des pays seront toujours stockés dans cette variable et nous pourrons y faire référence dès que nécessaire

country_labels = sparse_df3.loc[:]["Country Name"]

In [ ]:
# On enlèvre la colonnes 'Country Name' 
sparse_df3 = sparse_df3.drop(columns = ['Country Name'])

In [ ]:
corr_df = sparse_df3.corr()
corr_df[:5]

In [ ]:
# Représentation graphique des correlations
plt.figure(figsize= (15, 12))
sns.heatmap(corr_df,annot=True)
plt.show()

In [ ]:
sns.clustermap(corr_df,
               figsize= (16, 12),
               annot=True,
               dendrogram_ratio=(0.1, 0.2),
               row_cluster=False)
plt.show()

In [ ]:
def delete_corr(df, limit):
    #On construit notre matrice de correlation en valeur absolue
    corr_matrix = df.corr().abs()
    
    #Triangle supérieur de la matrice de corrélation : 
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > limit)]
    df = df.drop(df[to_drop], axis=1)
    return(df)

In [ ]:
df = delete_corr(sparse_df3, 0.6)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize= (15, 12))
sns.heatmap(df.corr().abs(), annot = True)
plt.show()

In [ ]:
df.index = country_labels

In [ ]:
df = df.drop(columns=["Net domestic credit (current LCU) [FM.AST.DOMS.CN]"])

In [ ]:
df.head()

# Normalisation :

In [ ]:
sns.pairplot(data=df)
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
sns.boxplot(data = df)
plt.show()

#On voit que les ordres de grandeur de nos données ainsi que leur variances ne sont pas du tout homogènes.
#On va donc les normaliser (centrer - réduire) pour corriger ce défaut  

In [ ]:
# Fonction qui centre et réduit une colonne de données pour les normaliser
def normalizer (data):
    # Quelques statistiques :
    mean = np.mean(data)
    var = np.var(data)
    sd = math.sqrt(var)
    
    norm = []
    for x in data:
        norm.append((x-mean)/sd)
    
    return (norm)

In [ ]:
def norm_whole_df (df):
    normed_df = df.copy(deep = True)
    for i in df.columns:
        normed_df[i] = normalizer(df[i])
        
    return (normed_df)

In [ ]:
normed_df = norm_whole_df(df)

In [ ]:
normed_df.head()

In [ ]:
plt.figure(figsize=(12,15))
sns.boxplot(data = normed_df)
plt.show()

In [ ]:
def search_outliers_row(df):
    tmp = df.copy(deep = True)
    for i in range (tmp.shape[0]):
        for j in range (tmp.shape[1]):
            if (tmp.iloc[i, j] > 3):
                df = df.drop(tmp.index[i])
    return (df)

In [ ]:
df = search_outliers_row(normed_df)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(12,15))
sns.boxplot(data = df)
plt.show()

In [ ]:
#Pairplot après standardisation
sns.pairplot(data=df)
plt.show()

## Clustering :

### Dataset clustering using K-means

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [ ]:
# Création d'un object KMeans :
km = KMeans(n_clusters=6, init='k-means++',n_init=10, random_state=50, max_iter=300,).fit(df)

In [ ]:
clusters = km.labels_

In [ ]:
# On stocke les centroides obtenus :
centroids=km.cluster_centers_

In [ ]:
predict = km.predict(df)

In [ ]:
print(predict)

In [ ]:
SSE=km.inertia_
print(SSE)

In [ ]:
# Using matplotlib.pyplot instead of seaborn.scatterplot to display the clusters.
plt.scatter(df["Death rate, crude (per 1,000 people) [SP.DYN.CDRT.IN]"], df["Aquaculture production (metric tons) [ER.FSH.AQUA.MT]"], c=clusters)
plt.title('Data in Space', fontsize=14)
plt.xlabel("Death rate, crude (per 1,000 people) [SP.DYN.CDRT.IN]",fontsize=14)
plt.ylabel("Aquaculture production (metric tons) [ER.FSH.AQUA.MT]",fontsize=14 )
plt.scatter(centroids[:, 3], centroids[:, 1], c='red',s=150, alpha=0.4)
plt.grid(True)

In [ ]:
# Using matplotlib.pyplot instead of seaborn.scatterplot to display the clusters.
plt.scatter(df["Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]"], df["Aquaculture production (metric tons) [ER.FSH.AQUA.MT]"], c=clusters)
plt.title('Data in Space', fontsize=14)
plt.xlabel("Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]",fontsize=14)
plt.ylabel("Aquaculture production (metric tons) [ER.FSH.AQUA.MT]",fontsize=14 )
plt.scatter(centroids[:, 0], centroids[:, 2], c='red',s=150, alpha=0.4)
plt.grid(True)

In [ ]:
# Visualisation du changement de nombre de cluster
km = KMeans(n_clusters=3, init='k-means++',n_init=10, random_state=50, max_iter=300,).fit(df)

SSE=km.inertia_
print(SSE)

In [ ]:
#Courbe de la SSE en fonction du nombre de clusters
SSE_liste=[]
for i in range(2,10):
    km2=KMeans(n_clusters=i, init='k-means++',  n_init=1, random_state=50, max_iter=1000).fit(df)
    SSE_liste.append(km2.inertia_)
#print(SSE_liste)

SSE_liste_random=[]
x=[]
for i in range(2,10):
    km3=KMeans(n_clusters=i, init='random',  n_init=1, random_state=50, max_iter=1000).fit(df)
    SSE_liste_random.append(km3.inertia_)
    x.append(i)
#print(SSE_liste_random)

plt.plot(x, SSE_liste,label="k-means++")
plt.plot(x, SSE_liste_random,label="Random")
plt.xlabel("Nombre de clusters")
plt.ylabel("SSE")
plt.legend()
plt.show()